In [1]:
## ensemble

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-01 02:55:44 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658526,35.2,1454558,77.7,1122311,60.0
Vcells,1222112,9.4,8388608,64.0,1975163,15.1


In [2]:
setwd("~")

In [3]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset1 <- fread("~/datasets/competencia_02_crudo.csv.gz" )
dataset2<- fread("~/datasets/competencia_03_crudo.csv.gz" )

dataset <- rbind(dataset1, dataset2)


# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-12-01 02:55:45 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,768049,41.1,1454558,77.7,1454558,77.7
Vcells,1301333804,9928.4,1777923864,13564.5,1449739655,11060.7


[1] "2025-12-01 02:56:18 UTC"

In [4]:
# 1) Traer dataset del mes y ganancia real
dfuture <- dataset[foto_mes == 202107]
dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]


# 2) Traer archivo de predicciones en test
files <- list.files("/home/guadalesandro/buckets/b1/exp",
                    pattern="^prediccion_mid\\.txt$",
                    recursive=TRUE,
                    full.names=TRUE)


# 3) leer todos los archivos con header=TRUE
pred_list <- lapply(files, function(f){
  fread(f, sep="\t", header=TRUE, encoding="UTF-8")
})

pred <- rbindlist(pred_list, use.names=TRUE, fill=TRUE)


# 4) Une la predicción y la ganancia real por número de cliente

pred[, numero_de_cliente := as.character(numero_de_cliente)]
dfuture[, numero_de_cliente := as.character(numero_de_cliente)]
pred[, prob := as.numeric(prob)]


pred <- merge(
  pred[, .(numero_de_cliente, prob)],
  dfuture[, .(numero_de_cliente, ganancia)],
  by = "numero_de_cliente",
  all.x = TRUE
)

# 4) Promedia la probabilidad por cliente
pred_ensamble <- pred[, .(
  prob_mean = mean(prob),
  ganancia = first(ganancia)
), by = numero_de_cliente]

# 5) Ordenar y ganancia acumulada
setorder(pred_ensamble, -prob_mean)
pred_ensamble[, gan_acum := cumsum(ganancia)]

# 6) Ganancia final de los 11000 más probables
envios <- 11500
ganancia_final <- pred_ensamble[envios, gan_acum]
ganancia_final

print(files)

[1] 422800000

 [1] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_b2/prediccion_mid.txt"
 [2] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_d/prediccion_mid.txt" 
 [3] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e/prediccion_mid.txt" 
 [4] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e2/prediccion_mid.txt"
 [5] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_f/prediccion_mid.txt" 
 [6] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_h/prediccion_mid.txt" 
 [7] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_j/prediccion_mid.txt" 
 [8] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k/prediccion_mid.txt" 
 [9] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k2/prediccion_mid.txt"
[10] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k3/prediccion_mid.txt"
[11] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k4/prediccion_mid.txt"
[12] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k5/prediccion_mid.txt"
[13] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_k6/prediccion_mid.txt"
[14] "/home/guadalesandro

In [5]:
# ====== 7) Métricas del ensamble ======

# Clase real: 1 si BAJA+2 (ganancia positiva)
pred_ensamble[, clase_real := ifelse(ganancia == 780000, 1, 0)]

# Clase predicha: 1 para los 11500 más probables
pred_ensamble[, clase_pred := 0]
pred_ensamble[1:envios, clase_pred := 1]

# Matriz de confusión
TP <- pred_ensamble[clase_real == 1 & clase_pred == 1, .N]
FP <- pred_ensamble[clase_real == 0 & clase_pred == 1, .N]
TN <- pred_ensamble[clase_real == 0 & clase_pred == 0, .N]
FN <- pred_ensamble[clase_real == 1 & clase_pred == 0, .N]

# Métricas
accuracy  <- (TP + TN) / (TP + TN + FP + FN)
precision <- TP / (TP + FP)
recall    <- TP / (TP + FN)
f1        <- 2 * precision * recall / (precision + recall)

# Mostrar resultados
list(
  ganancia_final = ganancia_final,
  accuracy  = accuracy,
  precision = precision,
  recall    = recall,
  f1        = f1,
  TP = TP, FP = FP, TN = TN, FN = FN
)


$ganancia_final
[1] 422800000

$accuracy
[1] 0.9324649

$precision
[1] 0.07095652

$recall
[1] 0.6538462

$f1
[1] 0.1280201

$TP
[1] 816

$FP
[1] 10684

$TN
[1] 152664

$FN
[1] 432

In [7]:
# ========= 7) Métricas por cada modelo individual ==========

# 7.1 volver a cargar cada archivo y agregar el nombre del modelo
pred_list <- lapply(files, function(f){
  df <- fread(f, sep="\t", header=TRUE, encoding="UTF-8")

  # nombre de la carpeta del experimento como identificador del modelo
  df[, modelo := basename(dirname(f))]

  df
})

pred_all <- rbindlist(pred_list, use.names=TRUE, fill=TRUE)

# 7.2 corregir tipos y unir con la verdad real
pred_all[, numero_de_cliente := as.character(numero_de_cliente)]
pred_all[, prob := as.numeric(prob)]

dfuture2 <- copy(dfuture)
dfuture2[, numero_de_cliente := as.character(numero_de_cliente)]

pred_all <- merge(
  pred_all,
  dfuture2[, .(numero_de_cliente, ganancia)],
  by = "numero_de_cliente",
  all.x = TRUE
)

# clase real binaria
pred_all[, clase_real := ifelse(ganancia == 780000, 1, 0)]

envios <- 11000

# 7.3 calcular métricas por cada modelo
metricas_modelo <- pred_all[
  , {
      # ordenar por prob en este modelo
      dt <- .SD[order(-prob)]

      # clase predicha según el top K
      dt[, clase_pred := 0]
      dt[1:envios, clase_pred := 1]

      TP <- dt[clase_real == 1 & clase_pred == 1, .N]
      FP <- dt[clase_real == 0 & clase_pred == 1, .N]
      TN <- dt[clase_real == 0 & clase_pred == 0, .N]
      FN <- dt[clase_real == 1 & clase_pred == 0, .N]

      accuracy  <- (TP + TN) / (TP + TN + FP + FN)
      precision <- TP / (TP + FP)
      recall    <- TP / (TP + FN)
      f1        <- 2 * precision * recall / (precision + recall)

      list(
        accuracy = accuracy,
        precision = precision,
        recall = recall,
        f1 = f1,
        TP = TP, FP = FP, TN = TN, FN = FN
      )
    },
  by = modelo
]

metricas_modelo


modelo,accuracy,precision,recall,f1,TP,FP,TN,FN
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
apo-505_C3_b2,0.9353447,0.07300000,0.6434295,0.1311234,803,10197,153151,445
apo-505_C3_d,0.9352232,0.07209091,0.6354167,0.1294905,793,10207,153141,455
apo-505_C3_e,0.9353326,0.07290909,0.6426282,0.1309602,802,10198,153150,446
apo-505_C3_e2,0.9353690,0.07318182,0.6450321,0.1314500,805,10195,153153,443
apo-505_C3_f,0.9353690,0.07318182,0.6450321,0.1314500,805,10195,153153,443
apo-505_C3_h,0.9353204,0.07281818,0.6418269,0.1307969,801,10199,153149,447
apo-505_C3_j,0.9353204,0.07281818,0.6418269,0.1307969,801,10199,153149,447
apo-505_C3_k,0.9354662,0.07390909,0.6514423,0.1327564,813,10187,153161,435
apo-505_C3_k2,0.9353569,0.07309091,0.6442308,0.1312867,804,10196,153152,444


In [ ]:
## predicción final ensamble

In [21]:
dfuture <- dataset[foto_mes == 202109]

In [28]:
# 1 - Levanta predicciones y las concatena
files <- list.files("/home/guadalesandro/buckets/b1/exp", #/C3
                    pattern="^prediccion\\.txt$",
                    recursive=TRUE,
                    full.names=TRUE)

print(files)

pred_list <- lapply(files, function(f){
  fread(f, sep="\t", header=TRUE, encoding="UTF-8")})
pred <- rbindlist(pred_list, use.names=TRUE, fill=TRUE)


pred[, numero_de_cliente := as.character(numero_de_cliente)]
pred[, prob := as.numeric(prob)]

#promedia probabilidades y las ordena
pred_ensamble <- pred[, .(prob_mean = mean(prob, na.rm=TRUE)), by = numero_de_cliente]
setorder(pred_ensamble, -prob_mean)

# Filtrar solo clientes de dfuture
clientes_future <- unique(dfuture$numero_de_cliente)
pred_ensamble <- pred_ensamble[numero_de_cliente %in% clientes_future]


envios <- 11500
pred_ensamble[, Predicted := 0L]
pred_ensamble[1:envios, Predicted := 1L]

cat("Cantidad de registros en el archivo final:", nrow(pred_ensamble), "\n")

#Guarda el archivo en carpeta final
output_dir <- "/home/guadalesandro/dmeyf2025/babycode/R_C3/C3_final"

# crear la carpeta si no existe
if(!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)
archivo_final <- file.path(output_dir, "C3_final_Alesandro.csv")

fwrite(
  pred_ensamble[, .(numero_de_cliente, Predicted)],
  file = archivo_final,
  sep = ","
)

cat("Archivo guardado en:", archivo_final, "\n")


 [1] "/home/guadalesandro/buckets/b1/exp/apo-505_a/prediccion.txt"                                                 
 [2] "/home/guadalesandro/buckets/b1/exp/apo-505_a2/prediccion.txt"                                                
 [3] "/home/guadalesandro/buckets/b1/exp/apo-505_base_integrada/prediccion.txt"                                    
 [4] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_b/prediccion.txt"                                              
 [5] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_c/prediccion.txt"                                              
 [6] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_d/prediccion.txt"                                              
 [7] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e/prediccion.txt"                                              
 [8] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_e2/prediccion.txt"                                             
 [9] "/home/guadalesandro/buckets/b1/exp/apo-505_C3_f/prediccion.txt"   